In [1]:
! pip3 install openpyxl xlrd

/bin/bash: pip3: command not found


In [1]:
import torch
import pandas as pd
import re

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, RandomSampler, DataLoader
from transformers import AutoConfig, AutoModel, AutoTokenizer
from tqdm import tqdm

/root/miniconda3/envs/myconda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tqdm.pandas(desc="process")

# load model
采用清华大学开源chatglm-6b模型的int8优化版本，显存需求不超过8G

In [3]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)

In [3]:
model = AutoModel.from_pretrained("THUDM/chatglm-6b-int8", trust_remote_code=True).half().cuda()
model = model.eval()

No compiled kernel found.
Compiling kernels : /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int8/22906aeb32fd7952ce323dc9d25e01693b270da6/quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int8/22906aeb32fd7952ce323dc9d25e01693b270da6/quantization_kernels_parallel.c -shared -o /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int8/22906aeb32fd7952ce323dc9d25e01693b270da6/quantization_kernels_parallel.so
Load kernel : /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int8/22906aeb32fd7952ce323dc9d25e01693b270da6/quantization_kernels_parallel.so
Setting CPU quantization kernel threads to 48
Using quantization cache
Applying quantization to glm layers


In [4]:
model.chat(tokenizer, '你好')

The dtype of attention mask (torch.int64) is not bool


('你好👋！我是人工智能助手 ChatGLM-6B，很高兴见到你，欢迎问我任何问题。',
 [('你好', '你好👋！我是人工智能助手 ChatGLM-6B，很高兴见到你，欢迎问我任何问题。')])

In [7]:
promp = """
你是一个导购机器人，这里有一篇笔记本电脑的产品描述：
#聊一款高性价比的入门级游戏本# #聊一款高性价比的入门级游戏本# #原创#猪王# 笔吧评测室 #2023-04-11 09:59#发表于#上海#收录于合集##机械革命#35#个##价格敏感型消费者#161#个# # # #今日点评【411】# #音频：# # # # # # # # # # # # # # #00:00# #00:27# # # # # # # # # # #说道“性价比游戏本”，很多人的第一反应是“机械革命”，但我们今天的主角并非机械革命 极光Pro，而是师出同门的“极光E”系列。#同样身为极光，极光E的性价比很高，#i7#+#RTX4050#售价#5499元#，是一款很便宜的入门级游戏本。#那么这台电脑实测表现究竟如何？#今天我们就来简单分析一下：#机械革命 极光E 极速版#左滑看接口#机身左侧#机身右侧#机身后部#它的配置如下：#i7-12650H 处理器#RTX4050#6GB 独立显卡（105W）#16GB DDR4#3200MHz 内存#512GB 固态硬盘#15.6英寸 2560×1440分辨率 100%sRGB色域 165Hz刷新率 IPS屏#厚\xa022.6~26.7mm#机身重\xa02.15kg#适配器重\xa0792g#参考售价5499元#它的优缺点如下：#优点！#1，性价比较高#2，同价位游戏本中屏幕素质较好#3，机身重量比较轻#缺点！#1，使用QLC固态硬盘#2，网卡性能较差#3，高负载下，键盘温度较高#【升级建议】#这台笔记本电脑拆机不难，卸下底面螺丝即可揭开后盖。#双通道16GB#DDR4 3200MHz内存能满足大部分用途的需求，如有需要可自行更换内存。#测试机的固态硬盘容量为512GB，型号是Intel 670P，QLC颗粒，支持PCIe3.0x4和NVMe，如有需要可自行加装固态硬盘。#无线网卡是Intel的AX101，支持1T1R和80MHz频宽，最大连接速率600MHz，吞吐性能不如2T2R的WiFi5#160MHz#（最大连接速率1733MHz）#，这对于内外网访问都会有不小的影响，如果有条件可以自行更换网卡。#【购买建议】#1，不想要太重的游戏本#2，对屏幕素质有一定要求#3，价格敏感型消费者#机械革命 极光E 极速版最大的特点就是便宜，i7+RTX4050仅售5499元。#屏幕方面#，它采用了一块经过出厂校色的2.5K165Hz屏，这一点待遇比极光Pro还更高。实测色域容积110.8%sRGB，色域覆盖99.4%sRGB，平均ΔE 1.29，最大ΔE 2.53，实测最大亮度412nit。#接口方面#，机身左侧依次为防盗锁口、USB2.0、耳机麦克风接口；#机身右侧为两个USB-A 5Gbps、SD卡槽#（2.0速率）#；#机身后侧为电源接口、RJ45网口、HDMI2.1、USB-C 10Gbps、miniDP接口。#噪音方面#，在环境噪音为36.9dB时，它的满载人位分贝值为53.5dB。#机械革命 极光E 极速版主要在PDD卖，就是一台专注于杀价的游戏本。#所以如果你的预算不足，就想买高性价比游戏本，那么这台电脑可以考虑一下。#但如果你想要一台各方面表现均衡的游戏本，那么这台电脑并不适合你。#【散热分析】#上图是机械革命 极光E 极速版的拆机实拍图，五热管双风扇的组合。#室温25℃#反射率1.0#BIOS版本：N.1.09MRO08#在满载状态下，开启狂暴模式，CPU温度最高96℃，稳定在91℃左右，功耗50W，P核频率3.0GHz，E核频率2.4GHz；#显卡功耗约105W，温度77.5℃，频率2265MHz。#如果单烤Stress#FPU，CPU温度维持在94℃，功耗82W，P核频率3.9GHz，E核频率3.0GHz。#如果单烤Furmark，显卡温度72.8℃，功耗105W，频率2250MHz。#表面温度如上图所示，键盘键帽最高#47℃#出现在键盘中部，WASD键附近约为39.2℃，方向键32℃。左腕托温度为26.5℃。#总的来说，机械革命 极光E 极速版的散热表现一般，主要问题在于键盘温度控制上，游戏常用的键区比较热，建议有条件的话外接键盘使用。#【猪王的良心结语】#机械革命 极光E 极速版的表现不出所料，作为主打性价比的系列，整体表现和极光 Pro十分接近。#适配器没有采用极光Pro那款小巧的SiC适配器，相对重一些。#在我看来，机械革命搞性价比最大的对手是他们自己：#前段时间我找到了这台机械革命 蛟龙15K，RTX4050游戏本居然仅售#4798元#就这价格还能给高色域屏，这简直是疯了啊……#客观地说，#电脑价格卖便宜了，就必须在某些地方缩水#，即便是机械革命也只能接受QLC硬盘和垃圾网卡。#正所谓#“姜太公钓鱼愿者上钩”#，价格都这么低了，相信真正有需求的用户会自己学着升级硬盘和网卡的#预览时标签不可点#收录于合集 ###机械革命#\xa035#个#上一篇#聊一款搭载RTX4050的大屏游戏本#下一篇#聊一台拥有12个大核心的游戏本#喜欢此内容的人还喜欢#微信扫一扫#关注该公众号#：#，#。#\xa0#视频#小程序#赞#，轻点两下取消赞#在看#，轻点两下取消在看#
客户诉求是：我想要一台外观材质较好、重量较轻、屏幕素质好的电脑
请你针对客户的诉求，介绍该产品的特点，注意只回复客户提及并且产品描述中有相应描述的部分
"""

In [8]:
model.chat(tokenizer, promp, max_length=4096)

('产品名称：机械革命 极光E 极速版\n\n产品描述：\n机械革命 极光E 极速版是一款入门级别的游戏本，它拥有i7-12650H处理器、RTX4050显卡、16GB DDR4内存、512GB固态硬盘和15.6英寸2560×1440分辨率IPS屏幕等特点。该电脑的外观设计材质较好，重量较轻，屏幕素质好，是一款非常适合想要一款外观材质较好、重量较轻、屏幕素质好的电脑的用户。\n\n优点：\n1. 外观材质较好：该电脑的外观设计材质较好，采用了高品质的金属机身和IPS屏幕，外观时尚、大气，非常适合追求外观的用户。\n2. 重量较轻：该电脑的机身重量较轻，只有2.15kg，非常适合携带，方便用户在日常生活中使用。\n3. 屏幕素质好：该电脑采用了2.5K分辨率的IPS屏幕，色域容积高达110.8%，色域覆盖也99.4%，屏幕素质非常好。\n4. 缺点：\n1. 使用QLC固态硬盘：该电脑的固态硬盘容量为512GB，型号是Intel 670P,QLC颗粒，支持PCIe3.0x4和NVMe，如有需要可以自行加装固态硬盘。\n2. 无线网卡性能较差：该电脑的无线网卡是Intel的AX101，支持1T1R和80MHz频宽，最大连接速率600MHz，吞吐性能不如2T2R的WiFi5#160MHz#(最大连接速率1733MHz)#，这对于内外网访问都会有不小的影响，如果有条件可以自行更换网卡。\n\n购买建议：\n1. 不想要太重的游戏本：如果预算不足，就想买高性价比游戏本，那么这台电脑可以考虑一下。\n2. 对屏幕素质有一定要求：如果对屏幕素质有一定要求，那么机械革命 极光E 极速版最大的特点就是便宜，i7+RTX4050仅售5499元。\n3. 价格敏感型消费者：机械革命 极光E 极速版最大的特点就是便宜，i7+RTX4050仅售5499元。\n\n【升级建议】\n这台笔记本电脑拆机不难，卸下底面螺丝即可揭开后盖。如果测试机的固态硬盘容量为512GB，型号是Intel 670P,QLC颗粒，支持PCIe3.0x4和NVMe，如有需要可以自行加装固态硬盘。如果升级到1TB的固态硬盘，那么价格会贵一些，但是性能会提升很多。\n\n【购买建议】',
 [('\n你是一个导购机器人，这里有一篇笔记本电脑的产品描述：\n#聊一款高性价比的入门级游戏本# #聊一款高性价比的入门级游戏本# 

# load data

In [4]:
data = pd.read_csv('./data/data.csv')
data = data.fillna('')
data.shape

(42, 21)

In [5]:
data.columns

Index(['file', 'ori_text', 'text', 'head', '升级建议', '购买建议', '散热分析', '猪王的良心结语',
       '游戏测试', '处理器分析', '显卡性能分析', '配置', '优缺点', '优点', '缺点', 'head_short',
       '屏幕方面', '接口方面', '噪音方面', '续航方面', 'suggestion_short'],
      dtype='object')

In [6]:
data['common'] = data[['head_short', '优缺点', 'suggestion_short', '猪王的良心结语']].agg('#'.join, axis=1)
data['common'] = data['common'].str.replace(r'[#\s]{2}', '#')

In [7]:
excel = pd.ExcelFile('./data/prompts.xls')
prompts = {x: pd.read_excel(excel, sheet_name=x) for x in excel.sheet_names}

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 2.0.1 for xls Excel support Use pip or conda to install xlrd.

# prompts and extract description

In [11]:
answers = {}
def get_propmt(feature):
    return f'你是一个信息抽取模型，你负责从一篇笔记本电脑的评测文章中帮我提取该产品的“{feature}”描述。如未找到相关描述则回复“未描述”。'

In [ ]:
for key, df in prompts.items():
    sample1, sample2 = df.at[1, 'sample'], df.at[2, 'sample']
    few_shot_history = [
        (df.at[0, 'sample'], f'好的, 我会帮你抽取该产品的“{key}”描述'),
        (sample1[:sample1.find('答案应该是')], re.search(r'答案应该是：?(.*)', sample1).group(1)),
        (sample2[:sample2.find('答案应该是')], re.search(r'答案应该是：?(.*)', sample2).group(1)),
    ]
    tqdm.pandas(desc=key)
    answers[key] = df['prompt'].progress_apply(lambda x: x and model.chat(tokenizer, x, history=few_shot_history, max_length=4096)[0])

售后:  98%|█████████▊| 41/42 [04:06<00:06,  6.38s/it]

In [ ]:
pd.DataFrame(answers).to_csv('./data/infos.csv', index=False)